# Experiments with GP pushing and then simulated annealing for finalization

In [2]:
from DLC_for_WBFM.utils.projects.finished_project_data import finished_project_data
import numpy as np
import matplotlib.pyplot as plt
import pickle
import napari, zarr

In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed tw

In [3]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
dat = finished_project_data.load_final_project_data_from_config(fname)

In [4]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\raw\match_dat.pickle"
with open(fname, 'rb') as f:
    matches = pickle.load(f)
    
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\raw\frame_dat.pickle"
with open(fname, 'rb') as f:
    frames = pickle.load(f)

C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator GaussianProcessRegressor from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# Visualize the pushed neurons on the next frame

In [5]:
pair = (0, 1)
print(matches[pair])
print(frames[pair[0]], frames[pair[1]])

FramePair with 134 matches 

                ReferenceFrame:
                Frame index: 0 
                Number of neurons: 148 
                ReferenceFrame:
                Frame index: 1 
                Number of neurons: 151 



In [6]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

In [7]:
# Get gp pushed locations

gp_zxy = matches[pair].gp_pushed_locations
# Replace z coordinate, which is somehow crazy
gp_zxy[:, 0] = frames[pair[0]].neuron_locs[:, 0]
gp_zxy = gp_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
# Replace z coordinate, which is somehow crazy
# affine_zxy[:, 0] = frames[pair[0]].neuron_locs[:, 0]
affine_zxy = affine_zxy[:, [0,2,1]]

In [8]:
# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

gp_xyz_matched = [row for i, row in enumerate(gp_zxy) if i in has_match_ind]
gp_xyz_unmatched = [row for i, row in enumerate(gp_zxy) if i not in has_match_ind]

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [9]:
# Plot using napari
v = napari.view_image(dat.red_data[pair[1]], ndisplay=3)
# v.add_points(gp_xyz_matched, size=3, face_color='green', n_dimensional=True)
v.add_points(gp_xyz_unmatched, size=3, face_color='red', n_dimensional=True)
# v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True)
v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

<Points layer 'Points [3]' at 0x21deeb64588>

In [ ]:

# Get gradient images (not in z)
# vol = dat.red_data[pair[1], ...]
# vol_filtered = gaussian(vol, sigma=1)
# thresh = 1.5*np.mean(vol_filtered)
# mask = vol_filtered > thresh
# vol_gradient_x = sobel_h(vol_filtered, mask)
# vol_gradient_y = sobel_v(vol_filtered, mask)

# Actually do it (no annealing)

In [10]:
# from skimage.filters import gaussian, sobel_h, sobel_v
from scipy.spatial import KDTree

In [11]:
pair = (1, 2)

In [12]:
# Get initial affine locations
affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Get image
vol = dat.red_data[pair[1], ...]
# vol_filtered = gaussian(vol, sigma=1)

# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

In [13]:
# Only check the ones with no match
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}
has_match_ind = list(f0_to_f1_matches.keys())
no_match_ind = list(range(frames[pair[0]].num_neurons()))
[no_match_ind.remove(i) for i in has_match_ind]

# Set up kdtree to check nearest neighbor
tree = KDTree(target_zxy)

# Parameters
brightness_thresh = 25
distance_thresh = 5.0

In [24]:
new_f1_zxy = []
new_matches = []
next_f1_ind = frames[pair[1]].num_neurons() + 1

for ind_f0 in no_match_ind:
    pushed_zxy = affine_zxy[ind_f0, :]
    z, x, y = int(pushed_zxy[0]), int(pushed_zxy[1]), int(pushed_zxy[2])
    brightness = vol[z, x, y]
    
    nn_dist, _ = tree.query(pushed_zxy, k=1)
    
    to_keep = (brightness > brightness_thresh) and (nn_dist > distance_thresh)
    print(to_keep, brightness, nn_dist)
    if to_keep:
        new_matches.append([ind_f0, next_f1_ind, 1.0])
        next_f1_ind += 1
        new_f1_zxy.append(pushed_zxy)

print(f"Added {len(new_matches)} neurons")

False 0 278.634677184079
False 22 6.514810434866421
True 40 8.798306660844792
False 31 3.65074508888712
False 34 4.273267236144742
True 36 9.445009640142503
True 27 12.186874485455414
True 35 7.80057086110525
Added 4 neurons


In [25]:
# Plot using napari
v = napari.view_image(dat.red_data[pair[1]], ndisplay=3)
v.add_points(new_f1_zxy, size=3, face_color='green', symbol='x', n_dimensional=True)
v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

<Points layer 'Points [3]' at 0x21df8569fc8>

In [28]:
f0, f1 = frames[pair[0]], frames[pair[1]]

In [35]:
len(f0.keypoint_locs)

151

# Use piecewise affine

In [4]:
pair = (0, 1)
print(matches[pair])
print(frames[pair[0]], frames[pair[1]])

FramePair with 134 matches 

                ReferenceFrame:
                Frame index: 0 
                Number of neurons: 148 
                ReferenceFrame:
                Frame index: 1 
                Number of neurons: 151 



In [5]:
m = matches[pair]
f0, f1 = frames[pair[0]], frames[pair[1]]

In [11]:
f0_to_f1 = m.get_f0_to_f1_dict()

In [47]:
src = np.array([f0.keypoint_locs[k,[2,1]] for k in f0_to_f1.keys()])
dst = np.array([f1.keypoint_locs[v,[2,1]] for v in f0_to_f1.values()])

In [48]:

tform = PiecewiseAffineTransform()
tform.estimate(src, dst)

True

In [49]:
src_transformed = tform.inverse(f0.keypoint_locs[:,[2,1]])
src_transformed = np.where(src_transformed==-1, np.nan, src_transformed)

src_transformed = np.hstack([f0.keypoint_locs[:,[0]], src_transformed]) #[:, [0,2,1]]

In [50]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [52]:
# Plot using napari
v = napari.view_image(dat.red_data[pair[0]], ndisplay=3)
v.add_points(src_transformed, size=3, face_color='red', n_dimensional=True)
v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True)
# v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

<Points layer 'Points [2]' at 0x23cef3a4e08>

# Scratch: sanity checks for the frames

In [36]:
fname = r"C:\dlc_stacks\Test_project\2-training_data\all_tracklets.h5"
import pandas as pd

df = pd.read_hdf(fname)

In [39]:
df

bodyparts   neuron0                                      neuron1              \
coords            z           x           y likelihood         z           x   
0          4.921292  424.082745  297.894046        1.0  5.907292  418.150000   
1          4.923492  407.928893  279.961296        1.0  5.734856  408.894737   

bodyparts                          neuron2              ...   neuron112  \
coords              y likelihood         z           x  ...           y   
0          279.964583        1.0  4.517820  433.077568  ...  266.299472   
1          261.281033        1.0  3.510288  412.983539  ...  222.409666   

bodyparts             neuron113                                     neuron114  \
coords    likelihood          z           x           y likelihood          z   
0                1.0  19.381144  353.493151  327.765512        1.0  21.385406   
1                1.0  17.900735  334.201471  287.945588        1.0  18.589015   

bodyparts                                     
coords              x           y likelihood  
0          379.461459  278.178828        1.0  
1          369.868687  243.525884        1.0  

[2 rows x 460 columns]

In [44]:
df.diff()[1:]


bodyparts   neuron0                                   neuron1            \
coords            z          x         y likelihood         z         x   
1          0.002201 -16.153852 -17.93275        0.0 -0.172436 -9.255263   

bodyparts                         neuron2             ...  neuron112  \
coords             y likelihood         z          x  ...          y   
1         -18.683551        0.0 -1.007532 -20.094029  ... -43.889806   

bodyparts            neuron113                                 neuron114  \
coords    likelihood         z         x          y likelihood         z   
1                0.0 -1.480409 -19.29168 -39.819923        0.0 -2.796391   

bodyparts                                  
coords            x          y likelihood  
1         -9.592773 -34.652945        0.0  

[1 rows x 460 columns]

In [56]:
all_neurons = list(df.columns.levels[0])
for n in all_neurons:
    zd = df.diff()[1:][n]['z']
    if all(abs(zd) > 2):
        print(n, list(zd))

neuron10 [3.493177957801862]
neuron108 [2.853986180347949]
neuron114 [-2.7963908094303775]
neuron15 [-2.8650967506389193]
neuron21 [-3.075268817204302]
neuron26 [-2.3528161778441987]
neuron37 [2.2387787912592696]
neuron40 [2.7618143527234444]
neuron42 [2.7250721741061525]
neuron44 [2.942191806542578]
neuron45 [-2.006387197933959]
neuron50 [2.8219336219336206]
neuron54 [-2.2871859861414876]
neuron55 [-3.189421109281737]
neuron58 [-2.0960522975374456]
neuron62 [-4.773072141094692]
neuron65 [-2.2753794711659125]
neuron67 [-2.0682962374138825]
neuron7 [-2.1460816593923617]
neuron77 [3.1148170157975557]
neuron78 [-2.2900887573964503]
neuron79 [2.3144520048602644]
neuron81 [-4.213115478518322]
neuron82 [-7.638868908594549]
neuron89 [-2.3382365197099766]
neuron99 [-2.5187500000000007]


In [58]:
np.quantile(np.arange(10), 0.9)

8.1